<a href="https://colab.research.google.com/github/mira-create/chlorine_model/blob/main/4_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup google colab file and import packages

In [78]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


In [79]:
import matplotlib.pyplot as plt
plt.rcParams["axes.grid"] = False
import numpy as np
import pandas as pd
import seaborn as sns
import re
import scipy
import scipy
from scipy import stats
from sklearn import preprocessing
from sklearn.preprocessing import PowerTransformer
from sklearn import linear_model
import networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from google.colab import files
import math


# Process Experimental Data

In [80]:
##Import Virus Data from Mira
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1WFAGeTDHBuTO8iMZu-kJ-aDjjZAVuKBIBNeQvOHssGk/edit#gid=0')
sheet = wb.sheet1
data = sheet.get_all_values()
df = pd.DataFrame(data)
df.columns = df.iloc[0]
experimental_data_Mira = df.iloc[1:]



In [81]:
##Import Virus Data from Kaming
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1-4_4vjJvusrTjH0fAhMcYC3gJi2FX-lwTX29hbrikqM/edit#gid=0')
sheet = wb.sheet1
data = sheet.get_all_values()
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df_Kaming = df.iloc[1:]

experimental_data_Kaming = df_Kaming[['kobs (new approach)', 'Sample ID']]
experimental_data_Kaming.columns  = ["kobs_kaming", "sample_ID"]



## Rename and select important columns

In [82]:
#extract rows from datasets that have attached label values
data = experimental_data_Mira.loc[experimental_data_Mira['data_for_modeling']=='1'].copy()

#rename important columns to ease further processing
data = data.rename(columns={"Virus name and strain": "virus_name", "Strain details": "strain","Temperature (C)":'temp', "Paper ID":'paper_ID',"Virus purification type":'purification_level', "kobs with new code":'k_obs'});
data = data.rename(columns={ "Chlorine Concentration" : "chlor_conc", "Sample ID" : "sample_ID","Year":"year", "Author_list_edit":"author_list", "Buffer description":"buffer_description", "average_kobs":"average_kobs_original"})

#fill in incomplete values for strain and add a space
data['strain'] = data['strain'].fillna('').str.lower()
#make virus name lowercase
data['virus_name'] = data['virus_name'].str.lower()

#combine virus name and strain columns
data['virus_name_strain'] = data['virus_name'].str.replace("[ \t]+$",'', regex = True)+ ' ' + data['strain']

#extract only important columns
data = data.loc[:, ["virus_name","virus_name_strain","strain","temp", "pH", "paper_ID","kobs_mira","purification_level", "chlor_conc", "sample_ID", "year", "author_list","buffer_type","corr_author","high_chloride", 'buffer_description']]
#include average_kobs_original if I'm trying to do a comparison

In [83]:
data.columns

Index(['virus_name', 'virus_name_strain', 'strain', 'temp', 'pH', 'paper_ID',
       'kobs_mira', 'purification_level', 'chlor_conc', 'sample_ID', 'year',
       'author_list', 'buffer_type', 'corr_author', 'high_chloride',
       'buffer_description'],
      dtype='object', name=0)

##Calculate average kobs from two reviewers

In [84]:
data_merge.columns

Index(['virus_name', 'virus_name_strain', 'strain', 'temp', 'pH', 'paper_ID',
       'kobs_mira', 'purification_level', 'chlor_conc', 'sample_ID', 'year',
       'author_list', 'buffer_type', 'corr_author', 'high_chloride',
       'buffer_description', 'kobs_kaming'],
      dtype='object')

In [85]:
data = pd.merge(data.copy(), experimental_data_Kaming, how = 'left',  left_on='sample_ID', right_on='sample_ID')

#convert reviewer data to numeric
data['kobs_mira'] = np.abs(data['kobs_mira'].apply(pd.to_numeric))
data['kobs_kaming'] = np.abs(data['kobs_kaming'].apply(pd.to_numeric))
#data['average_kobs_original'] = np.abs(data['average_kobs_original'].apply(pd.to_numeric))

data["average_kobs"] = (data['kobs_mira'] + data['kobs_kaming'])/2

#check if merge matches up with previously calculated average_kobs
#data2 = data.copy()[["kobs_mira", "kobs_kaming", "average_kobs_redo", "average_kobs"]]
#data2['diff'] = data2['average_kobs_redo'] - data2['average_kobs']
#data2[data2['diff']>.00005]


In [86]:
# check that only data points manually observed to be fine are greater than 10% different

kvalues = data.copy()
kvalues['diff'] = (  np.abs(( kvalues['kobs_mira'] - kvalues['kobs_kaming'] )) /kvalues['kobs_mira']  )*100
diff10 = kvalues[kvalues['diff']>10]

fine_sampleid = ["31-9", '11-2', '35-8', '35-17', '35-20', '36-1', '36-3', '36-4', '41-6',
                 '41-10', '46-1', '46-2', '46-3', '46-4', '46-5', '46-6', '51-8', '51-10',
                 '59-2', '59-13', '60-10', '63-2', '81-4', '35-6', '46-8', '46-12', '46-13',
                 '51-18', '64-1','78-1', '35-12', '35-16', '35-18', '71-3', '54-6', '35-10']

diff10[~diff10['sample_ID'].isin(fine_sampleid)]

,virus_name,virus_name_strain,strain,temp,pH,paper_ID,kobs_mira,purification_level,chlor_conc,sample_ID,year,author_list,buffer_type,corr_author,high_chloride,buffer_description,kobs_kaming,average_kobs,diff


In [87]:
len(data)

(len(data.paper_ID.unique()))

72

## Temperature and pH processing

In [88]:
#convert ID from string
data['paper_ID'] = data['paper_ID'].apply(pd.to_numeric)

#replace temp with middle of temp range, and set room temp to 20 (68C)
data['temp'] = data['temp'].str.replace('20-25','22.5').str.replace('25-28','26.5').str.replace('25-26','25.5').str.replace('23-24','23.5').str.replace('1-5','3')
data['temp'] = data['temp'].str.replace('room temperature','20')

#convert numeric data to string
data['temp'] = data['temp'].apply(pd.to_numeric)
data['pH'] = data['pH'].apply(pd.to_numeric)

#find rate constant in terms of HOCl
pka = 7.53
data['alpha_0'] = 1/(1+ pow(10, -1* pka)/pow(10,-1*data['pH']) )

## Author and buffer processing

In [89]:
data['buffer'] = data['buffer_type']

#make all buffers synthetic buffer
data['buffer'] = data['buffer'].str.replace('phosphate saline','synthetic_buffer').str.replace('acetate','synthetic_buffer').str.replace('borate','synthetic_buffer').str.replace('phosphate carbonate','synthetic_buffer')
data['buffer'] = data['buffer'].str.replace('calcium','synthetic_buffer').str.replace('carbonate','synthetic_buffer').str.replace('magnesium','synthetic_buffer').str.replace('phthlate','synthetic_buffer')
data['buffer'] = data['buffer'].str.replace('phosphate','synthetic_buffer')

#make natural and treated waters category
data['buffer'] = data['buffer'].str.replace('treated surface water','natural_or_treated_water').str.replace('tap water','natural_or_treated_water').str.replace('treated groundwater','natural_or_treated_water').str.replace('surface water','natural_or_treated_water')
data['buffer'] = data['buffer'].str.replace('treated water','natural_or_treated_water')

#make high organics category
data['buffer'] = data['buffer'].str.replace('lettuce wash water','high_organics').str.replace("wastewater","high_organics")



In [90]:
len(data)
data['buffer'].unique()

array(['synthetic_buffer', 'high_organics', 'natural_or_treated_water',
       'ultrapure'], dtype=object)

## Abbreviations and virus name cleaning

In [91]:
import re
#data processing of experimental data

experiment = data.copy()

#replace virus full name with abbreviation in experiment dataset
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace('poliovirus','pv')
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace('coxsackievirus','cv')
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace('echovirus','echo')
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace('human adenovirus','hadv')
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace('adenovirus','hadv')
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace('pepper mild mottle virus','pmmov')
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace('enterovirus','entero')
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace('hepatitis','hep')
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace('phi6','phi 6')

#remova trailing space
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace("[ \t]+$",'', regex = True)

#set all MS2 to be the same
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace("ms2.*",'ms2', regex = True)

#set all cvb5 faulkner to be the same
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace("faulkner.*",'faulkner', regex = True)

#remove unnecessart final identifyer
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace(" an mg845887",'').str.replace(" an mg845888",'').str.replace(" an mg845889",'')
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace(" mg845890",'').str.replace(" mg845891",'').str.replace(" mg845892",'')
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace(" mg845893",'').str.replace(" mg845894",'').str.replace(" mg845895",'')

#simplify name of pr772
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace("baa-769-b1",'')

#assume that all hadv2 is the same, since vr-846 is the same as strain 6
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace("vr-846",'strain 6').str.replace("strain 6","")

#remove dashes from cv
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace("b5-",'b5 ').str.replace("b4-",'b4 ').str.replace("b1-",'b1 ')

#assumption: all cv b3 is cv b3 nancy
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace("cv b3",'cv b3 nancy').str.replace("nancy nancy",'nancy')

#remove prd1 strain designation (only using one)
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace("dsm 19107",'')

#assumption: all phi6 is the same
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace("phi 6 atcc 21781-b1",'phi 6')

#remove fr strain designation, add bacteriophage to fr and ga
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace("atcc 15767-b1",'bacteriophage')
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace("ga(\W|$)",'ga bacteriophage', regex = True)

#replace h1n1 h1n1 strain a/pr/8/34 vr-1469 with simpler
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace("h1n1 strain a/pr/8/34 vr-1469",'h1n1 a/pr/8/34', regex = True)

#assumption: replace 77-1 cv b4 with cvb4 jvb (since this is a lab strain)
experiment["virus_name_strain"] = np.where(experiment["sample_ID"]=="77-1",'cv b4 jvb', experiment["virus_name_strain"])

#assumption: pv2 P712 is vaccine strain
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace("pv 2 p712",'pv 2 sabin', regex = True)

#assumption: all hadv 41 is hadv 41 tak
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace("hadv 41.*",'hadv 41 tak', regex = True)

#assumption: reovirus 3 lang is actually reovirus 1 lang, since the Lang strain is reovirus 1
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace("reovirus 3 lang",'reovirus 1 lang', regex = True)

#assumption: all murine norovirus that is not PR772 is mnv 1 (see virus info spreadsheet)
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace("murine norovirus(?!\s)",'murine norovirus 1', regex = True)

#assumption: manually assign cv B5 in Payment 1985 to Faulkner
experiment["virus_name_strain"] = np.where(experiment["sample_ID"]=="77-2",'cv b5 faulkner', experiment["virus_name_strain"])

#assumption: assume that all mnv except that specifically not designated is mnv 1
experiment['virus_name_strain'].str.replace("murine norovirus(?!\s)",'murine norovirus 1', regex = True)

#remove trailing spaces
experiment['virus_name_strain'] = experiment['virus_name_strain'].str.replace("[ \t]+$",'', regex = True)

#(sorted(experiment.virus_name_strain.unique()))


In [92]:
len(sorted(experiment.virus_name_strain.unique()))

82

# Process Virus Reference Data

In [93]:
#import virus data URL
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1F77o3NBR8Km1cG7MWmJGoyzkT1d2eqjXSYx8BIUb5pY/edit#gid=1024744371')
sheet = wb.worksheet('Virus Details Final')
data2 = sheet.get_all_values()
df = pd.DataFrame(data2)
df.columns = df.iloc[0]
virus_data = df.iloc[1:]

## Rename and select important columns


In [94]:
#rename important columns
virus_data = virus_data.rename(columns={"Virus Name": "virus_name", "Baltimore Class": "balt_class", "Genome shape":'shape', "Family":'family',"Genus":'genus', "Strain":'strain'})
virus_data = virus_data.rename(columns={"Structure":'structure',"Tail":'tail',"Isoelectric Point": 'iep', "Species":'species', "Symmetry":'symmetry', "Tail":'tail'})
virus_data['virus_name_strain'] = virus_data['virus_name'].str.replace("[ \t]+$",'', regex = True) + ' ' + virus_data['strain']
virus_data['min_diameter'] = pd.to_numeric(virus_data['min_diameter'])
virus_data['max_diameter']= pd.to_numeric(virus_data['max_diameter'])
virus_data['diameter'] = (virus_data['min_diameter'] + virus_data['max_diameter'])/2
virus_data['genome_length'] = pd.to_numeric(virus_data['genome_length'])

#take the average of min and max diameter to get diameter

reference = virus_data.loc[:, ["virus_name_strain","balt_class","family","genus","species","shape","structure","symmetry","tail","diameter","CG_content","genome_length","C_percentage", "G_percentage", "A_percentage", "T_percentage", "U_percentage", "T_is_0", "U_is_0"]]

#reference = virus_data.loc[:,"virus_name_strain","balt_class","family","genus","species","shape","iep","structure","symmetry","tail","diameter","CG_content","genome_length","C_percentage", "G_percentage", "A_percentage", "T_percentage", "U_percentage", "T_is_0", "U_is_0"]

#make colums lowercase
reference['virus_name_strain'] = reference['virus_name_strain'].str.lower()
reference['structure'] = reference['structure'].str.lower()

#remove trailing space
reference['shape']  = reference['shape'].str.replace("[ \t]+$",'', regex = True)


#replace other virus characteristics
reference['balt_class'] = reference['balt_class'].str.replace('-senseRNA','-ssRNA')



## Abbreviations and virus name cleaning

In [95]:
#data processing of reference data

#replace virus full name with abbreviation in experiment dataset
reference['virus_name_strain'] = reference['virus_name_strain'].str.replace('poliovirus','pv')
reference['virus_name_strain'] = reference['virus_name_strain'].str.replace('coxsackievirus','cv')
reference['virus_name_strain'] = reference['virus_name_strain'].str.replace('echovirus','echo')
reference['virus_name_strain'] = reference['virus_name_strain'].str.replace('human adenovirus','hadv')
reference['virus_name_strain'] = reference['virus_name_strain'].str.replace('adenovirus','hadv')
reference['virus_name_strain'] = reference['virus_name_strain'].str.replace('pepper mild mottle virus','pmmov')
reference['virus_name_strain'] = reference['virus_name_strain'].str.replace('enterovirus','entero')
reference['virus_name_strain'] = reference['virus_name_strain'].str.replace('hepatitis','hep')
reference['virus_name_strain'] = reference['virus_name_strain'].str.replace('phi6','phi 6')
reference['virus_name_strain'] = reference['virus_name_strain'].str.replace('none','')

#decided to use only one ms2, so i will remove identifyer
reference['virus_name_strain'] = reference['virus_name_strain'].str.replace("atcc 15597-b1",'')

#general mods
reference['virus_name_strain'] = reference['virus_name_strain'].str.replace("phi 6 phage",'phi 6')

reference['virus_name_strain'] = reference['virus_name_strain'].str.replace("2-85",'')

reference['virus_name_strain'] = reference['virus_name_strain'].str.replace("hep e virus",'hep e').str.replace("hep a virus",'hep a').str.replace("sar 55",'sar55')

reference['virus_name_strain'] = reference['virus_name_strain'].str.replace(" 7a,",'')

#assume all hadv2 is same
reference['virus_name_strain'] = reference['virus_name_strain'].str.replace("hadv 2 strain 6",'hadv 2')

#remove trailing space
reference['virus_name_strain'] = reference['virus_name_strain'].str.replace("[ \t]+$",'', regex = True)


In [96]:
#reference.to_excel("virus_information.xlsx")
#files.download("virus_information.xlsx")


In [97]:
sorted(np.unique(reference['virus_name_strain']))

['aichi virus 1',
 'bacteriophage b40-8',
 'cv a24 eh24/70',
 'cv a5 swartz',
 'cv a9',
 'cv a9 griggs',
 'cv a9 pb bozek',
 'cv b1 49683',
 'cv b1 conn-5',
 'cv b1 l071715',
 'cv b2',
 'cv b3 nancy',
 'cv b4 jvb',
 'cv b4 m063015',
 'cv b4 t051217',
 'cv b5',
 'cv b5 ea 80',
 'cv b5 faulkner',
 'cv b5 l030315',
 'cv b5 l060815',
 'cv b5 l061815',
 'cv b5 l070215',
 'cv b5 l070915',
 'cv b5 m063015',
 'echo 1 farouk',
 'echo 11 gregory',
 'echo 12 travis',
 'echo 29 jv-10',
 'echo 5 noyce',
 "echo 6 d'amori",
 'echo 7 wallace',
 'echo 9 vispo',
 'entero 70 ahc(j670/71)',
 'f2 bacteriophage',
 'feline calicivirus f9',
 'fr bacteriophage',
 'ga bacteriophage',
 'h1n1 a/pr/8/34',
 'h5n1 a/cambodia/408008/2005 clade 1',
 'h5n1 a/chicken/hong kong/d-0947/2006',
 'h5n1 a/whooperswan/mongolia/244/2005',
 'hadv 12 huie',
 'hadv 2',
 'hadv 3 g.b.',
 'hadv 40 dugan',
 'hadv 41 tak',
 'hadv 5',
 'hadv 7 gomen',
 'hadv 7a s-1058',
 'hep a hm175',
 'hep a mbb',
 'hep e sar55',
 'human rotavirus',
 

# Merge Datasets

In [98]:
#merge dataset, using experimental data as key
merged_dataset = pd.merge(experiment, reference, how = 'left',  left_on='virus_name_strain', right_on='virus_name_strain')

#drop extraneous virus name column
merged_dataset = merged_dataset.drop(columns=['virus_name', 'strain'])



In [99]:
#check merged dataset for missing values
print(merged_dataset[merged_dataset['buffer'].isnull()])
print(merged_dataset[merged_dataset['balt_class'].isnull()])

len(np.unique(merged_dataset['virus_name_strain']))



Empty DataFrame
Columns: [virus_name_strain, temp, pH, paper_ID, kobs_mira, purification_level, chlor_conc, sample_ID, year, author_list, buffer_type, corr_author, high_chloride, buffer_description, kobs_kaming, average_kobs, alpha_0, buffer, balt_class, family, genus, species, shape, structure, symmetry, tail, diameter, CG_content, genome_length, C_percentage, G_percentage, A_percentage, T_percentage, U_percentage, T_is_0, U_is_0]
Index: []

[0 rows x 36 columns]
Empty DataFrame
Columns: [virus_name_strain, temp, pH, paper_ID, kobs_mira, purification_level, chlor_conc, sample_ID, year, author_list, buffer_type, corr_author, high_chloride, buffer_description, kobs_kaming, average_kobs, alpha_0, buffer, balt_class, family, genus, species, shape, structure, symmetry, tail, diameter, CG_content, genome_length, C_percentage, G_percentage, A_percentage, T_percentage, U_percentage, T_is_0, U_is_0]
Index: []

[0 rows x 36 columns]


82

## Post Merge Processing

In [100]:
#process virus name columns
merged_dataset['virus_name_strain'] = merged_dataset['virus_name_strain'].str.replace('infectious pancreatic necrosis virus powder mill strain','ipnv')
merged_dataset['virus_name_strain'] = merged_dataset['virus_name_strain'].str.replace('infectious hematopoietic necrosis virus wenatchee river strain','ihnv')
merged_dataset['virus_name_strain'] = merged_dataset['virus_name_strain'].str.replace('h5n1 a/chicken/hong kong/d-0947/2006','h5n1 hk')
merged_dataset['virus_name_strain'] = merged_dataset['virus_name_strain'].str.replace('h5n1 a/whooperswan/mongolia/244/2005','h5n1 mongolia')
merged_dataset['virus_name_strain'] = merged_dataset['virus_name_strain'].str.replace('bacteriophage b40-8','b40-8')
merged_dataset['virus_name_strain'] = merged_dataset['virus_name_strain'].str.replace('cv a24 eh24/70','cv a24')
merged_dataset['virus_name_strain'] = merged_dataset['virus_name_strain'].str.replace('entero 70 ahc\\(j670/71\\)','entero 70')
merged_dataset['virus_name_strain'] = merged_dataset['virus_name_strain'].str.replace('f2 bacteriophage','f2')
merged_dataset['virus_name_strain'] = merged_dataset['virus_name_strain'].str.replace('ga bacteriophage','ga')
merged_dataset['virus_name_strain'] = merged_dataset['virus_name_strain'].str.replace('fr bacteriophage','fr')
merged_dataset['virus_name_strain'] = merged_dataset['virus_name_strain'].str.replace('feline calicivirus f9','fcv f9')

merged_dataset['virus_name_strain'] = merged_dataset['virus_name_strain'].str.replace('h1n1 a/pr/8/34','h1n1 pr')

merged_dataset['virus_name_strain'] = merged_dataset['virus_name_strain'].str.replace('hadv 7a s-1058','hadv 7a')

merged_dataset['virus_name_strain'] = merged_dataset['virus_name_strain'].str.replace('human rotavirus','hrv')
merged_dataset['virus_name_strain'] = merged_dataset['virus_name_strain'].str.replace('murine norovirus','mnv')
merged_dataset['virus_name_strain'] = merged_dataset['virus_name_strain'].str.replace('rhesus rotavirus','rrv')
merged_dataset['virus_name_strain'] = merged_dataset['virus_name_strain'].str.replace('reovirus','reo')
merged_dataset['virus_name_strain'] = merged_dataset['virus_name_strain'].str.replace('phi x-174 atcc 13706-b1','phi x-174')


<ipython-input-100-3d6104a9cbd6>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  merged_dataset['virus_name_strain'] = merged_dataset['virus_name_strain'].str.replace('entero 70 ahc\\(j670/71\\)','entero 70')


## Modify author list

In [101]:
def get_author_list(str_list):
    str_list = str_list.replace('[', "")
    str_list = str_list.replace(']', "")
    str_list = str_list.replace("'", "")
    return str_list


merged_dataset['corr_author'] = merged_dataset['corr_author'].apply(get_author_list)
#merged_dataset['corr_author'].value_counts()

In [102]:
# remove extraneous features from dataset
merged_dataset = merged_dataset[[f for f in merged_dataset.columns if f not in ['Unnamed: 0', 'kobs_mira','kobs_kaming', 'chlor_conc', 'shape', 'iep']]]
merged_dataset.head()

,virus_name_strain,temp,pH,paper_ID,purification_level,sample_ID,year,author_list,buffer_type,corr_author,...,diameter,CG_content,genome_length,C_percentage,G_percentage,A_percentage,T_percentage,U_percentage,T_is_0,U_is_0
0,cv b5 faulkner,20.0,7.5,2,2,2-1,2020,"['young_s', 'torrey_j', 'bachmann_v', 'kohn_t']",phosphate saline,kohn_t,...,30.0,47.77,7400.0,22.97297,24.7973,28.32432,0,23.90541,1,0
1,echo 11 gregory,20.0,7.5,2,2,2-2,2020,"['young_s', 'torrey_j', 'bachmann_v', 'kohn_t']",phosphate saline,kohn_t,...,30.0,48.145,7438.0,23.78328,24.36139,28.09895,0,23.75639,1,0
2,cv b5 faulkner,20.0,7.0,3,2,3-1,2020,"['shirasaki_n', 'matsushita_t', 'matsui_y', 'k...",phosphate saline,shirasaki_n,...,30.0,47.77,7400.0,22.97297,24.7973,28.32432,0,23.90541,1,0
3,cv b5 faulkner,20.0,7.0,3,2,3-2,2020,"['shirasaki_n', 'matsushita_t', 'matsui_y', 'k...",phosphate saline,shirasaki_n,...,30.0,47.77,7400.0,22.97297,24.7973,28.32432,0,23.90541,1,0
4,cv b5 faulkner,10.0,7.0,5,2,5-1,2019,"['wati_s', 'robinson_b', 'mieog_j', 'blackbear...",wastewater,keegan_a,...,30.0,47.77,7400.0,22.97297,24.7973,28.32432,0,23.90541,1,0


In [103]:
# Take log of the kobs
merged_dataset['log_average_kobs'] = np.log10(merged_dataset['average_kobs'])


## Scale Year

In [104]:
merged_dataset['year'] = merged_dataset['year'].str.replace('b', '')
np.sort(merged_dataset['year'].unique())

array(['1958', '1971', '1973', '1977', '1978', '1979', '1980', '1981',
       '1982', '1983', '1984', '1985', '1986', '1988', '1989', '1990',
       '1991', '1994', '1995', '1996', '1998', '2003', '2004', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2018', '2019', '2020'], dtype=object)

In [105]:
middle_year = float((1958 + 2020)/2)

merged_dataset['year_float'] = merged_dataset['year'].astype("float")
merged_dataset['year_int'] = merged_dataset['year'].astype("int")

merged_dataset['float_year_scaled'] = (merged_dataset['year_float'] - middle_year)/10
merged_dataset['int_year_scaled'] = (merged_dataset['year_int'] - middle_year)/10

In [106]:
#(sorted(experiment.virus_name_strain.unique()))

#Export Dataset

In [107]:
len(sorted(merged_dataset.virus_name_strain.unique()))

82

In [108]:
import os
os.getcwd()

'/content'

In [109]:
from google.colab import files

merged_dataset_test = merged_dataset.copy().fillna('').reset_index(drop = True)

values = [merged_dataset_test.columns.values.tolist()] + merged_dataset_test.values.tolist()

sheet = gc.create('merged_dataset_postprocessing.xlsx').sheet1
sheet.update(values)


#merged_dataset_test.to_csv("merged_dataset_test.csv")
#files.download("merged_dataset_test.csv")

{'spreadsheetId': '1eAUCumt3TgfcfcwnoWypl6_JzlucQQR1g2Jm8XehVAg',
 'updatedRange': 'Sheet1!A1:AK565',
 'updatedRows': 565,
 'updatedColumns': 37,
 'updatedCells': 20905}